In [5]:
import pandas as pd
import numpy as np
import pinecone
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
# from openai.embeddings_utils import get_embedding
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import openai

# Load environment variables (e.g., API keys)

In [ ]:
load_dotenv("key.env")
openai.api_key = os.getenv('OPENAI_API_KEY')

In [8]:
tabular_data = pd.read_csv("input_table.csv")
qa_data = pd.read_excel("QA_dataset_share.xlsx")

In [9]:
tabular_data.fillna("N/A", inplace=True)

# Fill null values in QA data
qa_data.fillna("N/A", inplace=True)

C:\Users\yaswi\AppData\Local\Temp\ipykernel_9728\1077243820.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  qa_data.fillna("N/A", inplace=True)


# Convert tabular data rows into retrievable text


In [ ]:
def convert_row_to_text(row):
    return ", ".join([f"{col} = {row[col]}" for col in tabular_data.columns])

tabular_data["retrievable_text"] = tabular_data.apply(convert_row_to_text, axis=1)

# Initialize Pinecone and configure index

In [ ]:
# Initialize Pinecone and configure index
index_name = "hackathon"
pc = Pinecone(api_key="")  # Replace with your actual Pinecone API key

# Check if the index exists; create if it doesn't
if index_name not in [idx.name for idx in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric='euclidean',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
pinecone_index = pc.Index(index_name)

In [14]:
def convert_row_to_text(row):
    return ", ".join([f"{col} = {row[col]}" for col in tabular_data.columns])

tabular_data["retrievable_text"] = tabular_data.apply(convert_row_to_text, axis=1)



In [15]:
def get_embedding(text, model="text-embedding-ada-002"):
    
    client = OpenAI()

    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding


#Storing embeddings in pinecone (takes time)

In [20]:
for i, row in tabular_data.iterrows():
    row_text = row["retrievable_text"]  # Assume you've created a column for retrievable text
    embedding = get_embedding(row_text)
    pinecone_index.upsert([(str(i), embedding)])

In [26]:
def retrieve_context(question):
    # Embed the question
    question_embedding = get_embedding(question, model="text-embedding-ada-002")
    
    # Query Pinecone for the top matches
    search_results = pinecone_index.query(vector=question_embedding, top_k=3, include_metadata=False)
    
    # Retrieve matching rows
    context = []
    for match in search_results["matches"]:
        index = int(match["id"])
        context.append(tabular_data.iloc[index]["retrievable_text"])
    return context

In [27]:
def generate_answer(question, context):
    prompt = f"""
    You are a data assistant. Answer the question based on the provided context.

    Question: {question}
    Context: {" ".join(context)}
    Answer:
    """
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [28]:
def process_user_query(question):
    # Retrieve relevant rows
    context = retrieve_context(question)
    
    # Generate the answer using LLM
    answer = generate_answer(question, context)
    
    # Save the result to an Excel file
    result_df = pd.DataFrame({
        "Question": [question],
        "Generated Answer": [answer],
        "Context": [context]
    })
    result_df.to_excel("predicted_answers.xlsx", index=False)
    
    return answer

In [31]:
user_question = "What is the highest gross income for Health and Beauty product line?"
answer = process_user_query(user_question)
print("Answer:", answer)

Answer: The highest gross income for the Health and Beauty product line is 15.3225.
